In [ ]:
# Author: Thaddeus Pellegrini (thaddeus.pellegrini@ibm.com)
# Created: Wed. March 1 2023
# Modified by Haoran Liao

In [ ]:
from datetime import datetime, timedelta
from qiskit import IBMQ
from collections import defaultdict
import json
from tqdm.notebook import tqdm_notebook

In [ ]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub="ibm-q-internal", group='deployed')

In [ ]:
backends =  ['ibmq_montreal']

In [ ]:
interval_hour = 3 # interval in hours, sampling frequency
intervals = 120 # intervals*interval_hour = sampling time
ref_point = datetime.now() # change this to some datetime object if you want to grab data from different timeframe

times = []
for i in range(intervals):
    past_time = ref_point - timedelta(hours=i * interval_hour)
    times.append(past_time)

Format of datastructure:
    
historical_data.json:

{ device_1: 
    {times: [list of strings, times at which data was sampled],
     t1: {qubit_0: [list of t1 times, index corresponds to 'times'], qubit_1: [], ..., qubit_n: []},
     t2: {qubit_0: [list of t2 times, index corresponds to 'times'], qubit_1: [], ..., qubit_n: []},
     readout_err: {qubit_0: [list of readout errors, index corr. to 'times'], qubit_1: [], ..., qubit_n: []}
     cnots: {edge_0_1: [list of cnot errors, index corr. to 'times'], ... edge_n_m: [...]}
     },
    
  device_2: {...},
    .
    .
    .
  device_n: {...}
}

In [ ]:
data = {be: {} for be in backends}

for be in backends:
    print(be)
    backend = provider.get_backend(be)
    qubits = range(len(backend.properties().qubits))
    t1 = defaultdict(list)
    t2 = defaultdict(list)
    readout_err = defaultdict(list)
    cnot_err = defaultdict(list)
    sx_err = defaultdict(list)
    x_err = defaultdict(list)
    
    for qubit in qubits:
        t1s = []
        t2s = []
        readout_errs = []
        sx_errs = []
        x_errs = []

        for time in tqdm_notebook(times[::-1], leave=True):
            try:
                prop = backend.properties(datetime=time)
                t1s.append(prop.t1(qubit=qubit))
            except:
                t1s.append(-1)

            try: t2s.append(prop.t2(qubit=qubit))
            except: t2s.append(-1)

            try: readout_errs.append(prop.readout_error(qubit=qubit))
            except: readout_errs.append(-1)

            try: sx_errs.append(prop.gate_error(gate='sx', qubits=qubit))
            except: sx_errs.append(-1)

            try: x_errs.append(prop.gate_error(gate='x', qubits=qubit))
            except: x_errs.append(-1)
            
        t1[qubit] = t1s
        t2[qubit] = t2s
        readout_err[qubit] = readout_errs
        sx_err[qubit] = sx_errs
        x_err[qubit] = x_errs
    
    try:
        edges = backend.configuration().coupling_map # For BackendV1 (ibmq_device)
    except:
        edges = backend.coupling_map # For BackendV2 (ibm_device)

    print('Getting CX errors...')
    for edge in edges:
        qubit_1 = edge[0]
        qubit_2 = edge[1]
        
        cnots = []
        for time in tqdm_notebook(times[::-1], leave=True):
            print(time.strftime("%d-%m-%YT%H:%M:%S%Z"))
            try:
                prop = backend.properties(datetime=time)
                cnots.append(prop.gate_error(gate='cx', qubits=(qubit_1, qubit_2)))
            except:
                cnots.append(-1)
            
        e = str(qubit_1)+"_"+str(qubit_2)                
        cnot_err[e] = cnots
        
    data[backend.properties().backend_name]['times']=[time.strftime("%d-%m-%YT%H:%M:%S%Z") for time in times[::-1]]
    data[backend.properties().backend_name]['t1']=t1
    data[backend.properties().backend_name]['t2']=t2
    data[backend.properties().backend_name]['readout_err']=readout_err
    data[backend.properties().backend_name]['cnot_err']=cnot_err
    data[backend.properties().backend_name]['sx_err']=sx_err
    data[backend.properties().backend_name]['x_err']=x_err

In [ ]:
with open("./device_params/ibmq_montreal_params.json", "w") as outfile:
    json.dump(data, outfile)

In [ ]:
with open("./ibmq_lima_params.json", "r") as file:
    d = json.load(file)

In [ ]:
d['ibmq_lima']['times']